In [2]:
%matplotlib inline
import numpy as np
import scipy as sp
from itertools import product
import json 
import csv
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [3]:

#dataframe for the finding API, will be merged with the shopping GetSingleItem
df_finding=pd.read_json("./data/completedItems.json")

df_getsingleitem = pd.read_json("./data/getSingleItem.json")
df_getsingleitem=df_getsingleitem.rename(columns = {'ItemID':'itemId'})

df_UserProfile=pd.read_json("./data/getUserProfile.json")


We define a function to expand cells that are actually dictionaries into new columns based on dict keys. This is needed for access to item and user ids to merge the dataframes from the different APIs.

In [4]:
"""
Function to take each cell in a column when the cell is actually a dictionary. 
Each value in the dictionary is added to a new column of the dataframe based on its key
""" 

#how about this guy!!!
def column_dict_expander(df,column):
    for key in df[column][0].keys():
        templist = []
        print key
        counter = 0
        for i in df.index:
            counter +=1
            dictionary = df[column][i]
            if isinstance(dictionary,dict):
                if key in dictionary.keys():
                    val = dictionary[key]
            else:
                val = 'NaN'
            templist.append(val)
        df[key] = templist
    df = df.drop(column, 1)
    return df

In [5]:
# This is to expand some of the columns that have dicitonaries of information.
# I go into each dictionary and make a new column based on each key in the dictionary
# with the function I defined above 


df_getsingleitem = column_dict_expander(df_getsingleitem,'Seller')
df_UserProfile = column_dict_expander(df_UserProfile,'User')

"""
for key in df_UserProfile['User'][0].keys():
    templist = []
    print key
    for i in df_UserProfile.index:
        if df_UserProfile['Ack'][i] == 'Success':
            val = df_UserProfile['User'][i][key]
        else:
            val = 'NaN'
        templist.append(val)
    df_UserProfile[key] = templist
    
    
    
for key in df_getsingleitem['Seller'][0].keys():
    templist = []
    print key
    counter = 0
    for i in df_getsingleitem.index:
        counter +=1
        if key in df_getsingleitem['Seller'][i].keys():
            if df_getsingleitem['Seller'][i][key] != 'NaN':
                val = df_getsingleitem['Seller'][i][key]
        else:
            val = 'NaN'
        templist.append(val)
    df_getsingleitem[key] = templist
"""

FeedbackRatingStar
UserID
FeedbackScore
PositiveFeedbackPercent
Status
PositiveFeedbackPercent
SellerBusinessType
NewUser
UserID
FeedbackRatingStar
FeedbackPrivate
FeedbackScore
FeedbackDetailsURL
SellerItemsURL
RegistrationSite
RegistrationDate


"\nfor key in df_UserProfile['User'][0].keys():\n    templist = []\n    print key\n    for i in df_UserProfile.index:\n        if df_UserProfile['Ack'][i] == 'Success':\n            val = df_UserProfile['User'][i][key]\n        else:\n            val = 'NaN'\n        templist.append(val)\n    df_UserProfile[key] = templist\n    \n    \n    \nfor key in df_getsingleitem['Seller'][0].keys():\n    templist = []\n    print key\n    counter = 0\n    for i in df_getsingleitem.index:\n        counter +=1\n        if key in df_getsingleitem['Seller'][i].keys():\n            if df_getsingleitem['Seller'][i][key] != 'NaN':\n                val = df_getsingleitem['Seller'][i][key]\n        else:\n            val = 'NaN'\n        templist.append(val)\n    df_getsingleitem[key] = templist\n"

In [6]:
##FOR MERGING THE TWO DATAFRAMES ABOUT ITEM


#turns item ID entries from 1-element lists to just the contents inside
df_finding['itemId'] = df_finding.itemId.apply(lambda r: r[0])
#convert type of item ID entries to ints
df_finding['itemId'] = df_finding.itemId.apply(lambda r: int(r))

df_items = pd.merge(df_finding, df_getsingleitem, on='itemId', how='inner')


In [7]:
## MERGING THE FIRST TWO WITH THE ONE ABOUT THE SELLER

df_total = pd.merge(df_items, df_UserProfile, on='UserID', how='inner')

In [8]:
df_total.head()
print len(df_total.index)

9629


**NEXT, we will expand some more columns and delete others that we just don't need to do our analysis. This could even come at the end once we figure out which aren't useful, but must remember to put it here.**

We make a few functions that will help clean up the data to put it in usable form. The purpose of each is outlined in the heading of the function. 

In [9]:
"""
This function removes the array brackets from entries that are 1 element arrays. We can't do most of the
manipulation we want without doing so.

Input: a list r. if the list is 1D, return the first element. Else return the whole list again
"""
def de_list(r):
    if type(r) == list and len(r) == 1:
        return r[0]
    elif type(r) == list and len(r) == 0:
        return None
    elif type(r) == list and len(r) > 1:
        return r
    elif type(r) !=list:
        return r
    

In [10]:
"""
Function to change all strings that are supposed to represent bools to actual bools. Thanks eBay for being more
of a pain in the butt!
"""

def to_bool(df,col):
    for i in df.index:
        if df.loc[i,col] == u'true' or df.loc[i,col] == 'true':
            df.loc[i,col] = True
        elif df.loc[i,col] == u'false' or df.loc[i,col] == 'false':
            df.loc[i,col] = False
    return

In [11]:
"""
Function to change Column to include just one desired value in a dictionary. For example, each cell in the CurrentPrice
column is actually a dictionary with some redundant entries irrelevant to column title. We only want 
one of those entries -- the one that is actually the current price of the item at the end of the auction.

tl;dr - getting just one value from a column and keeping that column name rather than expanding a whole dictionary.

Takes dataframe df and column from that df, col
"""

def dict_simplify(df,col, key):
    for i in df.index:
        if pd.isnull(df[col].values[i]) == False:
            df[col].values[i] = df[col].values[i][key]
        else:
            continue
    return 

In [12]:
## Here we employ the functions written above on a variety of columns to either expand a dictionary into
# multiple columns or simply simplify a column to just one value in a dictionary. We also create a new 
# columns related to the number of photos of the item posted by the seller.

df_total['nPhotos'] = df_total['PictureURL'].map(lambda r: len(r))

for col in df_total.columns:
    df_total[col] = df_total[col].apply(de_list)

##expand every key in dict into new column
df_total = column_dict_expander(df_total,'condition')
df_total = column_dict_expander(df_total,'listingInfo')
df_total = column_dict_expander(df_total,'primaryCategory')
df_total = column_dict_expander(df_total,'shippingInfo')
df_total = column_dict_expander(df_total,'ReturnPolicy')


##specific dict value assigned to column based on Key
dict_simplify(df_total,'CurrentPrice', u'Value')
dict_simplify(df_total,'BuyItNowPrice', u'Value')
dict_simplify(df_total,'MinimumToBid', u'Value')
dict_simplify(df_total,'ProductID', u'Value')
dict_simplify(df_total,'sellingStatus', u'sellingState')


for col in df_total.columns:
    df_total[col] = df_total[col].apply(de_list)  

##convert string bools to actual bools
to_bool(df_total,'returnsAccepted')
to_bool(df_total,'bestOfferEnabled')
to_bool(df_total,'topRatedListing')
to_bool(df_total,'buyItNowAvailable')
to_bool(df_total,'expeditedShipping')
to_bool(df_total,'oneDayShippingAvailable')


conditionId
conditionDisplayName
listingType
gift
bestOfferEnabled
startTime
buyItNowAvailable
endTime
categoryId
categoryName
expeditedShipping
shippingType
handlingTime
oneDayShippingAvailable
shipToLocations
ReturnsAccepted




## Now we will do some Feature engineering

This is to figure out which variables actually make a distinction on profitability

In [13]:
df_total.head()

,autoPay,charityId,country,discountPriceInfo,galleryPlusPictureURL,galleryURL,globalId,isMultiVariationListing,itemId,location,paymentMethod,postalCode,productId,returnsAccepted,sellingStatus,subtitle,title,topRatedListing,viewItemURL,AutoPay,AvailableForPickupDropOff,BestOfferEnabled,BidCount,BuyItNowAvailable,BuyItNowPrice,Charity,ConvertedBuyItNowPrice,ConvertedCurrentPrice,Country,CurrentPrice,DiscountPriceInfo,EligibleForPickupDropOff,EndTime,ExcludeShipToLocation,GalleryURL,GlobalShipping,HandlingTime,HighBidder,HitCount,IntegratedMerchantCreditCardEnabled,ListingStatus,ListingType,Location,LotSize,MinimumToBid,NewBestOffer,PaymentAllowedSite,PaymentMethods,PictureURL,PostalCode,...,QuantityThreshold,ReserveMet,SKU,ShipToLocations,Site,StartTime,Storefront,Subtitle,TimeLeft,Title,TopRatedListing,ViewItemURLForNaturalSearch,FeedbackRatingStar_x,UserID,FeedbackScore_x,PositiveFeedbackPercent_x,Ack,Build,Errors,FeedbackHistory,Timestamp,Version,Status,PositiveFeedbackPercent_y,SellerBusinessType,NewUser,FeedbackRatingStar_y,FeedbackPrivate,FeedbackScore_y,FeedbackDetailsURL,SellerItemsURL,RegistrationSite,RegistrationDate,nPhotos,conditionId,conditionDisplayName,listingType,gift,bestOfferEnabled,startTime,buyItNowAvailable,endTime,categoryId,categoryName,expeditedShipping,shippingType,handlingTime,oneDayShippingAvailable,shipToLocations,ReturnsAccepted
0,false,NaN,US,NaN,NaN,http://thumbs4.ebaystatic.com/m/mg6-FUGNDJgBSJ...,EBAY-US,false,131642735687,"South Salem,NY,USA",PayPal,10590,NaN,False,EndedWithSales,NaN,iPhone 5s-32gb-Space Gray-unlocked,False,http://www.ebay.com/itm/iPhone-5s-32gb-Space-G...,False,NaN,False,41,NaN,NaN,NaN,NaN,"{u'CurrencyID': u'USD', u'Value': 202.09}",US,202.09,NaN,NaN,2015-11-09T01:48:14.000Z,NaN,http://thumbs4.ebaystatic.com/pict/13164273568...,False,2,"{u'FeedbackRatingStar': u'Blue', u'FeedbackPri...",190,False,Completed,Chinese,"South Salem, New York",NaN,204.59,False,None,PayPal,[http://i.ebayimg.com/00/s/OTk5WDc0OQ==/z/lDQA...,10590,...,NaN,NaN,NaN,US,US,2015-11-02T01:48:14.000Z,NaN,NaN,PT0S,iPhone 5s-32gb-Space Gray-unlocked,NaN,http://www.ebay.com/itm/iPhone-5s-32gb-Space-G...,Yellow,jdwein1980,19,100.0,Success,E949_CORE_APILW_17769283_R1,NaN,"{u'NeutralFeedbackPeriods': [{u'Count': 0, u'P...",2015-12-07T23:40:31.429Z,949,Confirmed,100.0,Undefined,False,Yellow,False,19,http://feedback.ebay.com/ws/eBayISAPI.dll?View...,http://search.ebay.com/?sass=jdwein1980&amp;ht=-1,US,2004-03-25T21:44:54.000Z,3,3000,Used,Auction,false,False,2015-11-02T01:48:14.000Z,False,2015-11-09T01:48:14.000Z,9355,Cell Phones & Smartphones,True,Calculated,2,False,US,ReturnsNotAccepted
1,false,NaN,US,NaN,NaN,http://thumbs1.ebaystatic.com/m/mQAr2vyjNsA8_c...,EBAY-US,false,262126933108,"Moody,AL,USA",PayPal,35004,NaN,False,EndedWithSales,NaN,apple iphone 5s 32gb,False,http://www.ebay.com/itm/apple-iphone-5s-32gb-/...,False,NaN,False,4,NaN,NaN,NaN,NaN,"{u'CurrencyID': u'USD', u'Value': 260.0}",US,260.00,NaN,NaN,2015-11-09T01:47:58.000Z,NaN,http://thumbs1.ebaystatic.com/pict/26212693310...,False,3,"{u'FeedbackRatingStar': u'Blue', u'FeedbackPri...",32,False,Completed,Chinese,"Moody, Alabama",NaN,265.00,False,None,PayPal,[http://i.ebayimg.com/00/s/MTAwMFg3NTA=/z/djoA...,35004,...,NaN,NaN,NaN,US,US,2015-11-04T01:47:58.000Z,NaN,NaN,PT0S,apple iphone 5s 32gb,NaN,http://www.ebay.com/itm/apple-iphone-5s-32gb-/...,Purple,forwhlrcr1965,565,100.0,Success,E949_CORE_APILW_17769283_R1,NaN,"{u'NeutralFeedbackPeriods': [{u'Count': 24, u'...",2015-12-07T23:25:19.360Z,949,Confirmed,100.0,Private,False,Purple,False,566,http://feedback.ebay.com/ws/eBayISAPI.dll?View...,http://search.ebay.com/?sass=forwhlrcr1965&amp...,US,2008-03-16T20:31:23.000Z,2,3000,Used,Auction,false,False,2015-11-04T01:47:58.000Z,False,2015-11-09T01:47:58.000Z,9355,Cell Phones & Smartphones,True,Free,3,False,US,ReturnsNotAccepted
2,false,NaN,US,NaN,NaN,http://thumbs2.ebaystatic.com/m/maMtR-RvKQp5pu...,EBAY-US,false,252155964081,"Seattle,WA,USA",PayPal,98104,"{u'@type

We delete the columns that are irrelevant to our analysis. For example, DiscountPriceInfo tells us no information about our data, because none of our auctions have discounted prices.

In [14]:
#This is to add columns for the number of negative and positive reviews in the past 30 and 365 days

for key in df_total['FeedbackHistory'][0].keys():
    templist_30 = []
    templist_365 = []
    
    if key == u'NegativeFeedbackPeriods':
        for i in df_total.index:
            ##get only negative feedback period
            neg_dict_array = df_total['FeedbackHistory'][i][key]
            neg_dict_30period = neg_dict_array[2][u'PeriodInDays']
            neg_dict_30count = neg_dict_array[2][u'Count']
            neg_dict_365count = neg_dict_array[4][u'Count']
            templist_30.append(neg_dict_30count)
            templist_365.append(neg_dict_365count)
            #if neg_dict_array[]
            #print neg_dict_array
        df_total['NegFeedback30Day'] = templist_30
        df_total['NegFeedback365Day'] = templist_365
    if key == u'PositiveFeedbackPeriods':
        for i in df_total.index:
            ##get only positive feedback period
            pos_dict_array = df_total['FeedbackHistory'][i][key]
            pos_dict_30period = pos_dict_array[2][u'PeriodInDays']
            pos_dict_30count = pos_dict_array[2][u'Count']
            pos_dict_365count = pos_dict_array[4][u'Count']
            templist_30.append(pos_dict_30count)
            templist_365.append(pos_dict_365count)
        
        df_total['PosFeedback30Day'] = templist_30
        df_total['PosFeedback365Day'] = templist_365
df_total = df_total.drop('FeedbackHistory', 1)

In [15]:
del df_total['autoPay'], df_total['charityId'], df_total['discountPriceInfo'], df_total['galleryPlusPictureURL']
del df_total['galleryURL'], df_total['isMultiVariationListing'], df_total['location'], df_total['productId']
del df_total['subtitle'], df_total['viewItemURL'], df_total['Charity'], df_total['ConvertedBuyItNowPrice']
del df_total['ConvertedCurrentPrice'], df_total['Country'], df_total['DiscountPriceInfo'], df_total['ExcludeShipToLocation']
del df_total['GalleryURL'], df_total['LotSize'], df_total['PictureURL'], df_total['PaymentAllowedSite']
del df_total['SKU'], df_total['Site'], df_total['Storefront'], df_total['Subtitle'], df_total['TimeLeft']
del df_total['title'], df_total['ViewItemURLForNaturalSearch'], df_total['Build'], df_total['Ack'], df_total['Errors']
del df_total['FeedbackDetailsURL'], df_total['SellerItemsURL'], df_total['RegistrationSite'], df_total['RegistrationDate']
del df_total['conditionDisplayName'], df_total['listingType'], df_total['gift'], df_total['startTime'], df_total['endTime']
del df_total['categoryName'], df_total['BuyItNowAvailable'], df_total['handlingTime'], df_total['shipToLocations']
del df_total['ReturnsAccepted'], df_total['PrimaryCategoryIDPath'], df_total['PrimaryCategoryName'], df_total['categoryId']
del df_total['Timestamp'], df_total['Version'], df_total['PaymentMethods'], df_total['PostalCode'], df_total['TopRatedListing']
del df_total['EligibleForPickupDropOff'], df_total['FeedbackRatingStar_y'], df_total['FeedbackScore_y']
del df_total['PositiveFeedbackPercent_y'], df_total['returnsAccepted'], df_total['NewBestOffer']
del df_total['QuantityAvailableHint'], df_total['QuantitySold'], df_total['QuantitySoldByPickupInStore'], df_total['HighBidder']
del df_total['globalId'], df_total['IntegratedMerchantCreditCardEnabled'], df_total['PrimaryCategoryID']
del df_total['FeedbackRatingStar_x'], df_total['Status']

We then remove unnecessary information from each of the columns. For example, on CurrentPrice, we only want the value in the dictionary and not the whole dictionary. 

In [16]:
df_test = df_total.copy()

In [17]:
print df_total['shippingType'].unique()
df_test['ListingType'] = df_test['ListingType'].map({'Chinese': 0, 'FixedPriceItem': 1})

[u'Calculated' u'Free' u'Flat' u'FlatDomesticCalculatedInternational'
 u'FreePickup' u'CalculatedDomesticFlatInternational' u'NotSpecified']


In [18]:
def filterfunc(key,elt):
    if elt == key and key in elt:
        return 0
    else:
        return 1

In [19]:
#df['CurrentPrice'].values[3][u'Value']
import time

df_clean = df_total.copy() 


df_clean['ListingType'] = df_clean['ListingType'].map({'Chinese': 0, 'FixedPriceItem': 1})
df_clean['paymentMethod'] = df_clean['paymentMethod'].apply(lambda x: filterfunc('PayPal',x))
df_clean['country'] = df_clean['country'].apply(lambda x: filterfunc('US',x))
df_clean['sellingStatus'] =df_clean['sellingStatus'].apply(lambda x: 0 if x == 'EndedWithSales' else 1)
df_clean['AvailableForPickupDropOff'] = df_clean['AvailableForPickupDropOff'].apply(lambda x: 0 if np.isnan(x) else 1)
df_clean['BuyItNowPrice'] = df_clean['BuyItNowPrice'].apply(lambda x: 0 if np.isnan(x) else 1)

df_clean['ShipToLocations'] = df_clean['ShipToLocations'].apply(lambda x: filterfunc('US',x))
df_clean['SellerBusinessType'] = df_clean['SellerBusinessType'].apply(lambda x: 0 if x == 'Commercial' else 1)
df_clean['shippingType'] = df_clean['shippingType'].apply(lambda x: 0 if x == 'Free' else 1)
df_clean['QuantityThreshold'] = df_clean['QuantityThreshold'].apply(lambda x: 0 if np.isnan(x) else 1)

df_clean['conditionId'] = df_clean['conditionId'].map({'1000':0, '1500':1, '1750':2, '2000':3, '2500':4,'3000':5, '4000':6, '5000':7, '6000':8, '7000':9})

df_clean['EndTime'] = df_clean['EndTime'].apply(lambda x: np.datetime64(x))
df_clean['StartTime'] = df_clean['StartTime'].apply(lambda x: np.datetime64(x))

df_clean['auctionLength'] = (df_clean['EndTime'] - df_clean['StartTime']).astype('timedelta64[h]')

del df_clean['EndTime'], df_clean['StartTime']
#CurrentPrice, start-end, HandlingTime, MinimumToBid, FeedbackScore_x, PositiveFeedbackPercent_x
#conditionId, Quantity

#Possible Zero-Important: Quantity, nPhotos, FeedbackScore, PositiveFeedbackPercent_x


#because each auction we are dealing with is completed, we say
df_clean=df_clean.rename(columns = {'CurrentPrice':'FinalPrice', 'PositiveFeedbackPercent_x': 'PositiveFeedbackPercent',
                                   'FeedbackScore_x':'FeedbackScore'})

In [28]:
df_clean = df_clean[pd.isnull(df_clean.HandlingTime) == False]
counter = 0
for i in df_clean.index:
    if np.isnan(df_clean.HandlingTime[i]) == True:
        counter+=1
print counter
df_clean.head()

0


,country,itemId,paymentMethod,postalCode,sellingStatus,topRatedListing,AutoPay,AvailableForPickupDropOff,BestOfferEnabled,BidCount,BuyItNowPrice,FinalPrice,GlobalShipping,HandlingTime,HitCount,ListingStatus,ListingType,Location,MinimumToBid,ProductID,Quantity,QuantityThreshold,ReserveMet,ShipToLocations,Title,UserID,FeedbackScore,PositiveFeedbackPercent,SellerBusinessType,NewUser,FeedbackPrivate,nPhotos,conditionId,bestOfferEnabled,buyItNowAvailable,expeditedShipping,shippingType,oneDayShippingAvailable,NegFeedback30Day,NegFeedback365Day,PosFeedback30Day,PosFeedback365Day,auctionLength
0,0,131642735687,0,10590,0,False,False,0,False,41,0,202.09,False,2,190,Completed,0,"South Salem, New York",204.59,NaN,1,0,NaN,0,iPhone 5s-32gb-Space Gray-unlocked,jdwein1980,19,100.0,1,False,False,3,5,False,False,True,1,False,0,0,5,5,168
1,0,262126933108,0,35004,0,False,False,0,False,4,0,260.00,False,3,32,Completed,0,"Moody, Alabama",265.00,NaN,1,0,NaN,0,apple iphone 5s 32gb,forwhlrcr1965,565,100.0,1,False,False,2,5,False,False,True,0,False,0,0,3,32,120
2,0,252155964081,0,98104,0,False,False,0,False,3,0,152.50,False,2,168,Completed,0,"Seattle, Washington",155.00,168534287,1,0,NaN,1,Apple iPhone 5s - 32GB SPACE GRAY AT&T,15-alice,15,94.1,1,False,False,2,5,False,False,False,1,False,0,1,2,17,120
3,0,252163774777,0,98104,0,False,False,0,False,1,0,99.00,False,2,143,Completed,0,"Seattle, Washington",100.00,168553370,1,0,NaN,0,Apple iPhone 5s - 32GB - Silver (Verizon) Smar...,15-alice,15,94.1,1,False,False,7,9,False,False,True,0,False,0,1,2,17,120
4,0,252171170160,0,98104,0,False,True,0,False,5,0,96.00,False,2,357,Completed,0,"Seattle, Washington",97.00,168534287,1,0,NaN,0,Apple iPhone 5s - 32GB SPACE GRAY AT&T,15-alice,15,94.1,1,False,False,2,5,False,False,False,1,False,0,1,2,17,120


In [29]:
dftouse = df_clean.copy()
df_clean.to_csv('dftouse.csv', encoding='utf-8')

**We can now make some separator variables to make listings unique**

Certain parameters that are zero are can be significant indicators of specific characteristics of an auction. For example, if the HitCount is zero, that means nobody really bothered to interact with the auction at all.

In [22]:
ZERO_IMPORTANT_VARIABLES = ['BidCount', 'HitCount', 'nPhotos', 'FeedbackScore', 'PositiveFeedbackPercent_x',
                            'conditionId']
STANDARDIZABLE = ['FinalPrice','auctionLength', 'HandlingTime', 'MinimumToBid', 'FeedbackScore_x', 
                  'PositiveFeedbackPercent_x', 'Quantity']

To mark binary or indicator variables, we see which of the dataframe fall into this category

In [23]:
for v in dftouse.columns:
    #print v
    l=dftouse[v].unique()
    if len(l) <= 10:
        #print 'hi'
        print v, l

country [0 1]
paymentMethod [0 1]
sellingStatus [0 1]
topRatedListing [False True]
AutoPay [False True]
AvailableForPickupDropOff [0 1]
BestOfferEnabled [False True]
BuyItNowPrice [0 1]
GlobalShipping [False True]
HandlingTime [ 2.  3.  1.  0.  4.]
ListingStatus [u'Completed' u'Active']
ListingType [0 1]
QuantityThreshold [0 1]
ReserveMet [ nan   0.   1.]
ShipToLocations [0 1]
SellerBusinessType [1 0]
NewUser [False True]
FeedbackPrivate [False True]
conditionId [5 9 0 1 3 4]
bestOfferEnabled [False True]
buyItNowAvailable [False True]
expeditedShipping [True False]
shippingType [1 0]
oneDayShippingAvailable [False True]


In [24]:
dftouse.shape

(9627, 43)